In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from video_processor import VideoProcessor
import datetime
import os


In [3]:
# import os
# import pandas as pd

# DATASET_DIR = '../datasets/KTH/'

# # Function to get list of video clips and their labels
# def get_video_clips_and_labels(dataset_dir):
#     data = {'clip_path': [], 'label': []}
    
#     for root, dirs, files in os.walk(dataset_dir):
#         for dir_name in dirs:
#             label = dir_name
#             clips_dir = os.path.join(root, dir_name)
            
#             for clip_file in os.listdir(clips_dir):
#                 clip_path = os.path.join(clips_dir, clip_file)
#                 data['clip_path'].append(clip_path)
#                 data['label'].append(label)
    
#     return data

# # Create DataFrame and save to CSV
# data = get_video_clips_and_labels(DATASET_DIR)
# df = pd.DataFrame(data)
# csv_path = os.path.join(DATASET_DIR, 'dataset.csv')
# df.to_csv(csv_path, index=False)

# print(f"CSV file saved at: {csv_path}")


In [4]:
df = pd.read_csv('../datasets/KTH/dataset.csv')
df.head()

,clip_path,label
0,../datasets/KTH/boxing\person01_boxing_d1_unco...,boxing
1,../datasets/KTH/boxing\person01_boxing_d2_unco...,boxing
2,../datasets/KTH/boxing\person01_boxing_d3_unco...,boxing
3,../datasets/KTH/boxing\person01_boxing_d4_unco...,boxing
4,../datasets/KTH/boxing\person02_boxing_d1_unco...,boxing


In [5]:
df['label'].unique()

array(['boxing', 'handclapping', 'handwaving', 'jogging', 'running', 'walking'], dtype=object)

In [28]:
classwise_df = []

for i in df['label'].unique():
    classwise_df.append(df[df['label']==i].reset_index())

# Inference on Dataset

In [29]:
from ultralytics import YOLO
import cv2

In [30]:
model = YOLO('models/yolov8n-pose.pt') 

In [1]:
batch_size = 10
vp = VideoProcessor(model,batch_size, max_frames=200, img_sz=320)

keypoints_df = pd.DataFrame()
class_keypoints_df = pd.DataFrame()
for train_df in classwise_df:  #, punch_df[0:1]
    label = train_df['label'][0]
    # print(f"Training: {label}")
    batch, skipped_id = vp.process_video(train_df['clip_path'])
    batch = np.array(batch)[:,0,:].reshape(-1,batch_size , 34)
    class_keypoints_df = pd.DataFrame({ 'keypoints': batch.tolist(), 'label': label})
    keypoints_df = pd.concat([keypoints_df, class_keypoints_df])
    print(f'Skipped: {len(skipped_id)}')

# keypoints_df.columns = ['keypoints', 'label']

cv2.destroyAllWindows()

NameError: name 'VideoProcessor' is not defined

In [35]:
keypoints_df

,keypoints,label
0,"[[0.47522578239440916, 0.1922487735748291, 0.4...",boxing
1,"[[0.496325159072876, 0.18797346750895183, 0.50...",boxing
2,"[[0.4961265563964844, 0.1917237917582194, 0.50...",boxing
3,"[[0.4779681205749512, 0.1823760668436686, 0.48...",boxing
4,"[[0.484557056427002, 0.18348032633463543, 0.49...",boxing
...,...,...
1995,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",walking
1996,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",walking
1997,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",walking
1998,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",walking


In [36]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'keypoint_dataset/KTH/keypoints_df_{timestamp}.csv'

# Save the keypoints_df to the new file
keypoints_df.to_csv(filename, index=False)

In [37]:
np.array(keypoints_df['keypoints'].tolist()).shape

(11980, 10, 34)

In [39]:
keypoints_df['label'].value_counts()

label
boxing          2000
handwaving      2000
jogging         2000
running         2000
walking         2000
handclapping    1980
Name: count, dtype: int64